In [1]:
import pandas as pd
import numpy as np
from collections import OrderedDict, deque, Counter
import matplotlib.pyplot as plt
from tqdm import tqdm
from random import randint
import random

random.seed(6)

ЧТОБЫ ПОЛУЧИТЬ 40 БАЛЛОВ, НЕОБХОДИМО СОЗДАТЬ ТАКОЙ "РЕЦЕПТ", КОТОРЫЙ В СРЕДНЕМ БУДЕТ ПРИНОСИТЬ 3 ЕДИНИЦЫ РАДОСТИ. ЕСЛИ ВАШ РЕЦЕПТ ПРИНОСИТ МЕНЬШЕ РАДОСТИ, ТО ВЫ ЗАРАБОТАЕТЕ МЕНЬШЕ ОЧКОВ, НО НЕ НОЛЬ :)

In [2]:
# загрузим данные с диска
history = pd.read_csv("history.csv")
example_submit = pd.read_csv("example_answer.csv")

In [3]:
example_submit.head()

,state,action0,action1,action2
0,393a1,34,33,33
1,2c674,34,33,33
2,eb3be,34,33,33
3,5aca6,34,33,33
4,9512d,34,33,33


In [4]:
print(history.shape)
history.head()

(1813, 4)


,state,action,reward,next_state
0,785f3,1,-5,785f3
1,0c75c,0,-1,da287
2,aa820,2,-1,aa820
3,e52d0,1,-2,e52d0
4,3055e,1,-2,3055e


In [5]:
ind2state = {}
state2ind = {}

# сформируем вспомогательные словари индексов и состояний
for i, state in enumerate(np.unique(history.state.values)):
    ind2state[i] = state
    state2ind[state] = i

In [6]:
# класс окружения (дневник Даши), с которым будет взаимодействовать агент

class environment:
    def __init__(self, history):
        self.states = history['state'].unique()
        self.env = {}
        
        # преобразуем историю Даши к словарю вида: 
        # состояние: {номер_действия: [детерминированная_вероятность_перехода, следующее_состояние, вознаграждение], ...}, ...
        for state in tqdm(self.states):
            next_states = history[history['state'] == state].drop_duplicates()

            self.env[state] = {}
            try:
                self.env[state][0] = [1.0, next_states[next_states['action'] == 0]['next_state'].values[0], int(next_states[next_states['action'] == 0]['reward'].values)]
            except:
                self.env[state][0] = [0.0, state, -100]

            try:
                self.env[state][1] = [1.0, next_states[next_states['action'] == 1]['next_state'].values[0], int(next_states[next_states['action'] == 1]['reward'].values)]
            except:
                self.env[state][1] = [0.0, state, -100]

            try:
                self.env[state][2] = [1.0, next_states[next_states['action'] == 2]['next_state'].values[0], int(next_states[next_states['action'] == 2]['reward'].values)]
            except:
                self.env[state][2] = [0.0, state, -100]

        self.cur_state = self.states[0]

    def reset(self, state=None):
    # необходимо реализовать функцию, которая будет изменять текущее состояние случайным образом на одно из возможных состояний (если state != None, то заменяет текущее состояние на аргумент функции)
        if state != None:
            self.cur_state = state
        else:
            self.cur_state = ind2state[np.random.randint(0, len(self.states))]

        return self.cur_state

    def sample(self):
        # функция, имитирующая случайное блуждание по состояниям
        p0 = self.env[self.cur_state][0][0]
        p1 = self.env[self.cur_state][1][0]
        p2 = self.env[self.cur_state][2][0]

        if p0 == 0:
            action = np.random.choice(np.arange(0, 3), p=[0., 0.5, 0.5])
        elif p1 == 0:
            action = np.random.choice(np.arange(0, 3), p=[0.5, 0., 0.5])      
        elif p2 == 0:
            action = np.random.choice(np.arange(0, 3), p=[0.5, 0.5, 0.])
        else:
            action = np.random.choice(np.arange(0, 3), p=[0.34, 0.33, 0.33])

        return action

    def step(self, action):
    # реализуйте функцию, которая изменяет состояние и возвращает вознаграждение в зависимости от текущего состояния и предпринятого действия агента. Не забудьте изменить состояние, в котором находится агент
        reward = self.env[self.cur_state][action][-1]
        next_state = self.env[self.cur_state][action][-2]

        self.reset(next_state)

        return next_state, reward

env = environment(history)

100%|███████████████████████████████████████████████████████████████████████████████| 610/610 [00:03<00:00, 160.99it/s]


In [7]:
import numpy as np

# создайте рецепт, который агент будет составлять на основе дневника. Он представляет собой таблицу, как example_submit, инициализированную нулями.
# агент должен будет в процессе обучения выучить вероятности действий, которые надо выполнять, находясь в конкретном состоянии, для получения максимального вознаграждения.
# Такая таблица как раз и нужна Маше, чтобы знать, что делать и в какой ситуации.
q_table = np.zeros([610, 3])

In [8]:
"""Обучение агента"""

# Маше очень грустно, тк всё, что она сейчас имеет, это таблица равновероятных действий в любой ситуации. Скорее обучите агента!
import random
from IPython.display import clear_output

def egreedy_policy(q_table, state, epsilon=0.1):  
    if np.random.random() < epsilon:
        return np.random.choice(3)
    else:
        return np.argmax(q_table[state2ind[state]])

# Гиперпараметры
# Исследуйте и пробуйте!
alpha = 0.3 # 
gamma = 0.99 # 
epsilon = 0.3 # 

for i in tqdm(range(1, 60001)): # не изменяйте количество итераций по дневнику
    state = env.reset()
    
    epochs, reward, = 0, 0
    while epochs < 100: # не изменяйте количество переходов

        # Choose action        
        action = egreedy_policy(q_table, state, epsilon=epsilon)
        
        # Do the action
        next_state, reward = env.step(action)
        
        # Update q_table 
        td_target = reward + gamma * np.max(q_table[state2ind[next_state]])
        td_error = td_target - q_table[state2ind[state]][action]
        q_table[state2ind[state]][action] += alpha * td_error
        
        # Update state
        state = next_state

        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

100%|███████████████████████████████████████████████████████████████████████████| 60000/60000 [03:22<00:00, 296.06it/s]

Episode: 60000
Training finished.



In [9]:
"""Посмотрим какое вознаграждение в среднем нам дает случайное блуждание"""
total_penalties = 0
episodes = 100

for state in tqdm(env.states):
    state = env.reset(state)
    for _ in range(episodes):
        epochs, penalties, reward = 0, 0, 0
        
        while epochs < 100:
            action = env.sample()
            state, reward = env.step(action)

            epochs += 1

            penalties += reward

        total_penalties += penalties

# в среднем оно приносит -3 единицы радости, т.е. Маша становится несчастна после такого рецепта. Нам он не подходит :(
print(f"Average rewards per episode: {total_penalties / (len(env.states) * episodes * 100)}")

100%|████████████████████████████████████████████████████████████████████████████████| 610/610 [04:38<00:00,  2.19it/s]

Average rewards per episode: -3.1842003278688527


In [10]:
"""Посмотрим, какое среднее вознаграждение получается, если использовать q-таблицу"""
total_penalties = 0
episodes = 100

for state in tqdm(env.states):
    state = env.reset(state)
    for _ in range(episodes):
        epochs, penalties, reward = 0, 0, 0
        while epochs < 100:
            action = np.argmax(q_table[state2ind[state]])
            state, reward = env.step(action)

            epochs += 1

            penalties += reward

        total_penalties += penalties

# Чем больше, тем лучше! Но на 3 единицах радости можно будет остановиться
print(f"Average rewards per state: {total_penalties / (len(env.states) * episodes * 100)}")

100%|████████████████████████████████████████████████████████████████████████████████| 610/610 [00:37<00:00, 16.44it/s]

Average rewards per state: 3.9453060655737704


In [18]:
# сделайте так, чтобы вероятность наиболее вероятного действия равнялась 100, а остальные действия занулите
submission = pd.DataFrame(q_table, columns=['action0', 'action1', 'action2'])
submission['state'] = submission.index
submission['state'] = submission['state'].replace(ind2state)
submission = submission[['state','action0', 'action1', 'action2']]

In [20]:
submission.to_csv('Q_table.csv', index=False)

In [22]:
submission[submission.columns[1:]] = submission[submission.columns[1:]].apply(lambda x: round((x / sum(x))*100), axis=1).astype("int")
submission.to_csv('answer.csv', index=False)
submission

,state,action0,action1,action2
0,0038d,34,33,33
1,0087c,35,34,32
2,01299,34,33,33
3,02176,34,33,33
4,02384,34,33,33
...,...,...,...,...
605,fc56d,34,33,33
606,fc71f,34,33,33
607,fdb99,34,33,33
608,fde3f,34,33,33


# Детерминированный алгоритм переходов

In [23]:
marks = submission[submission.columns[1:]].apply(lambda x: [100 if p == max(x) else 0 for p in x], axis=1)

action_0, action_1, action_2 = [], [], []
for mark in marks:
    action_0.append(mark[0])
    action_1.append(mark[1])
    action_2.append(mark[2])
    
submission["action0"] = action_0
submission["action1"] = action_1
submission["action2"] = action_2

submission.to_csv('determine_submit.csv', index=False)

# ответ должен быть такого формата:
submission.head()

,state,action0,action1,action2
0,0038d,100,0,0
1,0087c,100,0,0
2,01299,100,0,0
3,02176,100,0,0
4,02384,100,0,0
